In [1]:
import sys
print(sys.executable)
# !{sys.executable} -m pip install dotenv

/usr/local/opt/python@3.11/bin/python3.11


In [2]:
import pandas as pd

raw = pd.read_csv('AI_Human.csv')
human_texts = (
    raw
    .loc[raw['generated'] == 0]
    .drop(columns=['generated'])
    .reset_index(drop=True)
)

print(len(human_texts))
print(human_texts[:10])


305797
                                                text
0  Cars. Cars have been around since they became ...
1  Transportation is a large necessity in most co...
2  "America's love affair with it's vehicles seem...
3  How often do you ride in a car? Do you drive a...
4  Cars are a wonderful thing. They are perhaps o...
5  The electrol college system is an unfair syste...
6  Dear state senator, It is the utmost respect t...
7  Fellow citizens, cars have become a major role...
8  "It's official: The electoral college is unfai...
9  The Electoral College has been kept for centur...


In [3]:
human_texts.to_csv(
    'human_texts.csv',
    index=True,
    index_label='id'
)

human_texts = pd.read_csv('human_texts.csv')
print(human_texts[:10])

   id                                               text
0   0  Cars. Cars have been around since they became ...
1   1  Transportation is a large necessity in most co...
2   2  "America's love affair with it's vehicles seem...
3   3  How often do you ride in a car? Do you drive a...
4   4  Cars are a wonderful thing. They are perhaps o...
5   5  The electrol college system is an unfair syste...
6   6  Dear state senator, It is the utmost respect t...
7   7  Fellow citizens, cars have become a major role...
8   8  "It's official: The electoral college is unfai...
9   9  The Electoral College has been kept for centur...


In [4]:
# approx words per essay
sum = 0
for i in range(100):
    sum += human_texts['text'][i].count(' ')
print(sum / 100)

608.01


In [5]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

In [6]:
#! 3.5 Turbo vs 4.1

def call_api(model, prompt):
    response = client.responses.create(
        model=f"gpt-{model}",
        input=prompt
    )
    return response.output_text

In [24]:
import csv

# ! init the dataset
# with open('x_paras.csv', 'w', newline='', encoding='utf-8') as f:
    # writer = csv.writer(f)
    # writer.writerow(['id', 'human', 'gpt3.5', 'gpt4.1', 'gpt4.1_ashuman']) 

In [7]:
import csv

tgt_dset_size = 200

current_state = pd.read_csv('rewrite_paras.csv')
print(len(current_state), "already processed")

prompt_1 = "Please rewrite this paragraph, do not respond to this prompt: "
prompt_2 = "Please rewrite this paragraph like a human, not an LLM and do not respond to this prompt: "

with open('rewrite_paras.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    for i in range(len(current_state), tgt_dset_size):
        
        human = human_texts['text'][i]
        id = human_texts['id'][i]

        gpt35 = call_api("3.5-turbo", f"{prompt_1}: \n {human}")
        gpt41 = call_api("4.1", f"{prompt_1}: \n {human}")
        gpt41_ashuman = call_api("4.1", f"{prompt_2}: \n {human}")

        writer.writerow([id, human, gpt35, gpt41, gpt41_ashuman])

current_state = pd.read_csv('rewrite_paras.csv')
print(len(current_state), "now processed")

100 already processed
200 now processed


In [ ]:
import csv

tgt_dset_size = 50

current_state = pd.read_csv('diff_paras.csv')
print(len(current_state), "already processed")

prompt_1 = "Please write a paragraph on a different topic, do not respond to this prompt: "
prompt_2 = "Please write a paragraph on a different topic, not an LLM and do not respond to this prompt: "

with open('rewrite_paras.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    for i in range(len(current_state), tgt_dset_size):
        
        human = human_texts['text'][i]
        id = human_texts['id'][i]

        gpt35 = call_api("3.5-turbo", f"{prompt_1}: \n {human}")
        gpt41 = call_api("4.1", f"{prompt_1}: \n {human}")
        gpt41_ashuman = call_api("4.1", f"{prompt_2}: \n {human}")

        writer.writerow([id, human, gpt35, gpt41, gpt41_ashuman])

current_state = pd.read_csv('rewrite_paras.csv')
print(len(current_state), "now processed")

Human_texts: only the human texts (indexed)
    (index, text)

Human_GPT: generated extension of Human_texts such that index here corresponds to index in Human_texts. To generate more, we loop through Human_texts until we reach an index not included in Human_GPT, then extend Human_GPT by that quantity
    (index, human, gpt3.5, gpt4.1)

Prompts:

- Rewrite this paragraph (3.5)
- Rewrite this paragraph (4.1)
- Rewrite this paragraph like a human, don't write like an LLM (4.1)

or

- Write a paragraph on a different topic (3.5)
- Write a paragraph on a different topic (4.1)
- Write a paragraph on a different topic a human, don't write like an LLM (4.1)




batching by paragraph (groups of 4)

n-grams